In [1]:
!pip install transformers accelerate langchain



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
pip install transformers langchain langchain-community


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%pwd

'c:\\Code\\End-to-End-Medical-Chatbot-Generative-AI\\research'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
def load_pdf_file(data):
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [7]:
import os
print(os.getcwd())


c:\Code\End-to-End-Medical-Chatbot-Generative-AI\research


In [8]:
extracted_data = load_pdf_file(data='../Data/')




In [9]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [10]:
text_chunks = text_split(extracted_data)
print('Length of text chunks:', len(text_chunks))

Length of text chunks: 5961


In [9]:
pip install -U langchain-huggingface


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
from langchain_huggingface import HuggingFaceEmbeddings 

def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings


In [11]:
pip install sentence-transformers


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
embeddings = download_hugging_face_embeddings()


c:\Code\End-to-End-Medical-Chatbot-Generative-AI\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
from dotenv import load_dotenv
import os

load_dotenv() 

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
print("PINECONE_API_KEY:", PINECONE_API_KEY)
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
print("GROQ_API_KEY:", GROQ_API_KEY)

PINECONE_API_KEY: pcsk_26nrdP_QV56uem8Nzz7KisdZPp3Xhi3REBBatL3AyGTxBwGP64XSpLXoFxy6mbLmzd4kMq
GROQ_API_KEY: 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


In [13]:
index_name = "medicalbot"  # or whatever name you want

from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)

# Delete index if it exists
if index_name in [index_info["name"] for index_info in pc.list_indexes()]:
    pc.delete_index(index_name)

# Create a new index
pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

# Connect to index
index = pc.Index(index_name)


In [2]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

In [14]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

retriever = docsearch.as_retriever()


In [15]:
query = "What are the symptoms of diabetes?"
results = docsearch.similarity_search(query, k=3)  # top 3 relevant chunks

for i, res in enumerate(results):
    print(f"Result {i+1}:\n{res.page_content}\n")


Result 1:
affected and can range greatly.
• Type I diabetes mellitus. Characterized by fatigue and
an abnormally high level of glucose in the blood
(hyperglycemia).
• Amyotrophic lateral schlerosis. First signs are stum-
bling and difficulty climbing stairs. Later, muscle
cramps and twitching may be observed as well as
weakness in the hands making fastening buttons or
turning a key difficult. Speech may become slowed or
slurred. There may also be difficluty swallowing. As

Result 2:
begin to fall. A person with diabetes mellitus either does
not make enough insulin, or makes insulin that does not
work properly. The result is blood sugar that remains
high, a condition called hyperglycemia.
Diabetes must be diagnosed as early as possible. If
left untreated, it can damage or cause failure of the eyes,
kidneys, nerves, heart, blood vessels, and other body
organs. Hypoglycemia, or low blood sugar, may also be
discovered through blood sugar testing. Hypoglycemia is

Result 3:
Resources
BOOKS


In [16]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name = index_name,
    embedding = embeddings,
)



In [17]:
docsearch

In [17]:
pip install transformers langchain langchain-community


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
#from langchain_community.llms import Ollama

#llm = Ollama(model="gemma:2b")


In [4]:
#from langchain_core.prompts import PromptTemplate
#from langchain.chains.combine_documents import create_stuff_documents_chain
#from langchain.chains import create_retrieval_chain

# Create your prompt
#prompt = PromptTemplate(
    #input_variables=["context", "input"],
    #template=(
        #"You are an assistant for medical question-answering tasks.\n"
        #"Use the following context to answer the question concisely. "
        #"If you don't know the answer, say so.\n\n"
        #"Context:\n{context}\n\nQuestion:\n{input}"
    #)
#)

# Stuff documents into prompt
#question_answer_chain = create_stuff_documents_chain(llm, prompt)

# Create the retrieval-based QA chain
#rag_chain = create_retrieval_chain(retriever, question_answer_chain)

# Ask the bot a question
#response = rag_chain.invoke({"input": "What is hypertension?"})
#print(response["answer"])


In [30]:
from langchain_core.prompts import PromptTemplate
from langchain_groq import ChatGroq
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

# Load your retriever (this could be FAISS, ChromaDB etc.)
# retriever = ...

# Define the LLM
llm = ChatGroq(temperature=0.7, model_name="llama3-70b-8192")

# Define Prompt
prompt = PromptTemplate(
    input_variables=["context", "input"],
    template=(
        "You are a knowledgeable and helpful medical assistant. "
        "Use the context below to provide a clear and informative answer to the question. "
        "If relevant, explain causes, symptoms, treatments, or examples. Do not skip any important detail.\n\n"
        "Context:\n{context}\n\n"
        "Question:\n{input}\n\n"
        "Answer:"
    ),
)


# Create RAG Chain
qa_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, qa_chain)

# Use the chain
response = rag_chain.invoke({"input": "What is hypertension?"})
print(response["answer"])


Hypertension, also known as high blood pressure, is a medical condition characterized by persistently high arterial blood pressure. In this condition, the force of blood pushing against the walls of the arteries is consistently too high, which can lead to damage to the blood vessels, heart, and kidneys over time.

Hypertension is often referred to as a "silent killer" because it usually does not cause noticeable symptoms until it has reached a severe stage or has caused complications such as heart disease, stroke, or kidney disease. However, some people may experience symptoms such as:

* Palpitations (rapid, forceful, throbbing, or fluttering heartbeat)
* Headaches
* Dizziness or lightheadedness
* Nosebleeds
* Fatigue

If left uncontrolled, hypertension can lead to serious complications, including:

* Heart failure: High blood pressure can weaken the heart, making it less efficient at pumping blood, leading to heart failure.
* Stroke: High blood pressure can cause blood vessels in the

In [ ]:
#question_answer_chain = create_stuff_documents_chain(llm, prompt)
#rag_chain = create_retrieval_chain(retriever, question_answer_chain)


In [31]:
response = rag_chain.invoke({"input": "what is Acromegaly and gigantism?"})
print(response["answer"])


Acromegaly and gigantism are related disorders caused by the abnormal release of growth hormone from the pituitary gland in the brain.

**Acromegaly:**
Acromegaly is a rare disorder that occurs when the pituitary gland releases excess growth hormone after normal bone growth has stopped, typically in middle-aged adults. This excess hormone causes increased growth in bone and soft tissue, leading to a range of symptoms and disturbances throughout the body. The symptoms of acromegaly develop gradually, which often leads to delayed diagnosis. The disorder affects approximately 50 out of every one million people, and both men and women are equally affected.

**Gigantism:**
Gigantism, on the other hand, occurs when the abnormal release of growth hormone occurs before normal bone growth has stopped, typically during childhood or adolescence. This results in abnormal growth and height, often leading to unusual tallness.

**Common symptoms of acromegaly and gigantism:**

* Increased growth in b

In [23]:
response = rag_chain.invoke({"input": "What is ac?"})
print(response["answer"])

Based on the context, "ac" likely refers to Acupuncture.
